# Groq + Exa MCP: Real-Time Web Search
This notebook shows Python developers how to connect Groq with Exa MCP's web search tools. Exa MCP helps AI models get current information from the internet. Exa provides smart web search that can find relevant content.

We will do this in three simple steps:
1. Set up **Groq client** for fast AI responses.
2. Set up **Exa MCP server** for web search and content crawling.
3. **Connect them together** using the Responses API.

---


## Getting Started

Follow these steps to set up:
1. **Sign up** for Exa at [dashboard.exa.ai](https://dashboard.exa.ai/api-keys) to get your free API key.
2. **Sign up** for Groq at [console.groq.com](https://console.groq.com/keys) to get your free API key.
3. **Copy your API keys** from your Exa and Groq account dashboards.
4. **Paste your API keys** into the cell below and run the cell.


In [ ]:
# To export your API keys into a .env file, run the following cell (replace with your actual keys):
!echo "EXA_API_KEY=<your-exa-api-key>" >> .env
!echo "GROQ_API_KEY=<your-groq-api-key>" >> .env


In [ ]:
import json
import os
import time
from datetime import datetime

# API Configuration
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
EXA_API_KEY = os.getenv("EXA_API_KEY")

# Check if API keys are set
if not GROQ_API_KEY:
    print("Please set your Groq production API key")
else:
    print("Groq API key configured successfully!")
if not EXA_API_KEY:
    print("Please set your Exa API key")
else:
    print("Exa API key configured successfully!")


Choose the AI model for our responses. We'll use OpenAI's [gpt-oss-120b](https://console.groq.com/docs/model/openai/gpt-oss-120b) model, which runs very fast on Groq.


In [ ]:
# Model configuration
MODEL = "openai/gpt-oss-120b"


## Step 1: Set up the Groq client


In [ ]:
from openai import OpenAI

# set up Groq MCP client
client = OpenAI(base_url="https://api.groq.com/api/openai/v1", api_key=GROQ_API_KEY)


## Step 2: Set up Exa's remote MCP server


In [ ]:
# set up Exa MCP server
tools = [
    {
        "type": "mcp",
        "server_url": f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}",
        "server_label": "exa",
        "require_approval": "never",
    }
]


## Step 3: Connect Groq to the Exa MCP through Groq's OpenAI-compatible Responses API


In [ ]:
def connect_groq_to_exa(client, tools, query):
    """
    Connect Groq to Exa for web search.

    This function shows how to combine:
    - Groq's fast AI responses
    - Exa's web search tools
    """

    start_time = time.time()

    # Call Groq with Exa MCP integration using responses API
    response = client.responses.create(
        model=MODEL,
        input=query,
        tools=tools,
        stream=False,
        temperature=0.1,
        top_p=0.4,
    )

    total_time = time.time() - start_time

    # Get response content from responses API
    content = (
        response.output_text if hasattr(response, "output_text") else str(response)
    )

    # collect executed tools (MCP tool calls)
    executed_tools = []

    # Extract MCP calls from response if available
    if hasattr(response, "output") and response.output:
        for output_item in response.output:
            if hasattr(output_item, "type") and output_item.type == "mcp_call":
                executed_tools.append(
                    {
                        "type": "mcp",
                        "arguments": getattr(output_item, "arguments", "{}"),
                        "output": getattr(output_item, "output", ""),
                        "name": getattr(output_item, "name", ""),
                        "server_label": getattr(output_item, "server_label", ""),
                    }
                )
    return {
        "content": content,
        "total_time": total_time,
        "mcp_calls_performed": executed_tools,
        "timestamp": datetime.now().isoformat(),
    }


Let's create a helper function to show what tools were used and their results. This helps us see exactly what Exa did behind the scenes.


In [ ]:
def print_mcp_calls(mcp_calls):
    executed_tools = mcp_calls["mcp_calls_performed"]
    if executed_tools:
        print(f"\nEXA MCP CALLS: Found {len(executed_tools)} tool call(s):")
        print("-" * 50)
        for i, tool in enumerate(executed_tools, 1):
            print(f"\nTool Call #{i}")
            print(f"   Type: {tool['type']}")
            print(f"   Tool Name: {tool['name']}")
            print(f"   Server: {tool['server_label']}")
            try:
                if tool["arguments"]:
                    args = (
                        json.loads(tool["arguments"])
                        if isinstance(tool["arguments"], str)
                        else tool["arguments"]
                    )
                    print(f"   Arguments: {args}")

                if tool["output"]:
                    output_data = (
                        json.loads(tool["output"])
                        if isinstance(tool["output"], str)
                        else tool["output"]
                    )
                    print(f"   Output: {str(output_data)[:500]}...")
            except Exception as e:
                print(f"   Could not parse tool data: {e}")
                
    print(f"   Total time: {mcp_calls['total_time']:.2f} seconds")
    print(f"   Exa MCP calls: {len(mcp_calls['mcp_calls_performed'])}")


## Available Exa Tools

Exa MCP provides several powerful tools for different types of research and search:

| Tool | Description |
|------|-------------|
| **`web_search_exa`** | Smart web search that finds relevant content using AI |
| **`company_research`** | Research companies by crawling their official websites |
| **`crawling`** | Extract content from specific URLs (articles, PDFs, web pages) |
| **`linkedin_search`** | Search LinkedIn for companies and people |
| **`deep_researcher_start`** | Start an AI researcher for complex questions with multiple sources |
| **`deep_researcher_check`** | Check if deep research is complete and get the report |


---


## Demo 1: Web Search


In [ ]:
from IPython.display import Markdown

web_search_demo = connect_groq_to_exa(
    client,
    tools,
    "Find recent breakthroughs in quantum computing research",
)


Let's display the agent's response in markdown format.


In [ ]:
Markdown(web_search_demo["content"])


Let's examine the agent's intermediate steps, including how it uses Exa's search capabilities.


In [ ]:
print_mcp_calls(web_search_demo)


## Demo 2: Try it Yourself

Now it's your turn! Replace the query with some real-time information you want to discover using Exa's search capabilities.


In [ ]:
your_query = "Your Query Here"  # Change this!

custom_response = connect_groq_to_exa(client, tools, your_query)


In [ ]:
Markdown(custom_response["content"])


In [ ]:
print_mcp_calls(custom_response)


## Summary

You've just experienced the power of combining **Groq's fast responses** with the **Exa Web Search MCP**.

## 🔍 **Exa Web Search MCP**

**Description**: Exa MCP is a powerful web search and web crawling MCP. It lets you do real-time web searches, extract content from any URL, and even run deep research for detailed reports.

**Resources**:
- 📖 **Documentation**: [https://docs.exa.ai/reference/exa-mcp](https://docs.exa.ai/reference/exa-mcp)
- 🔧 **Open Source**: [https://github.com/exa-labs/exa-mcp-server/](https://github.com/exa-labs/exa-mcp-server/)


### Available Tools:
1. **`web_search_exa`** - Web search for realtime and correct info
2. **`company_research`** - Business research by crawling websites
3. **`crawling`** - Extract content from specific URLs
4. **`linkedin_search`** - Professional network research
5. **`deep_researcher_start`** - Start comprehensive research report
6. **`deep_researcher_check`** - Get the completed research report

---

**Resources:**
- Explore the [Exa MCP documentation](https://docs.exa.ai/reference/exa-mcp)
- View the Exa MCP [opensource code](https://github.com/exa-labs/exa-mcp-server/)
- Check out the [Exa website](https://exa.ai)
